# Keras를 사용한 ML(머신러닝) 기본사항
## 6. 저장 및 로드
### - 모델 저장과 복원
#### 참고사이트 : https://www.tensorflow.org/tutorials/keras/save_and_load?hl=ko

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0'

In [3]:
# MNIST 데이터셋으로 모델을 훈련하여 가중치를 저장하는 예제
# 모델 실행 속도를 위해 샘플 중 처음 1000개만 사용
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [4]:
# 모델 정의
# 가중치를 저장하고 불러오는 예제를 위해 간단한 모델 생성
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

# 모델 객체 생성
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [7]:
# 훈련하는 동안 체크 포인트 저장하기
# 훈련 중간과 마지막에 checkpoint를 자동으로 저장하는 것이 많이 사용됨
# 다시 훈련하지 않고 모델을 재사용하거나, 훈련 과정이 중지된 경우 이어서 훈련 진행 가능

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# checkpoint callback 만들기(checkpoint 작업을 조정할 수 있도록 여러 매개변수 제공)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련(학습) 단계에 콜백을 전달합니다

# optimizer의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있음.
# 이 경고는 이전 사용 방식을 권장하지 않기 위함이며 무시해도 괜찮음.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 608/1000 [=================>............] - ETA: 0s - loss: 1.4991 - accuracy: 0.5674 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 598us/sample - loss: 1.2022 - accuracy: 0.6610 - val_loss: 0.7166 - val_accuracy: 0.7860
Epoch 2/10
 672/1000 [===================>..........] - ETA: 0s - loss: 0.4392 - accuracy: 0.8705
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 154us/sample - loss: 0.4328 - accuracy: 0.8730 - val_loss: 0.5532 - val_accuracy: 0.8230
Epoch 3/10
 672/1000 [===================>..........] - ETA: 0s - loss: 0.3000 - accuracy: 0.9196
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 152us/sample - loss: 0.2897 - accuracy: 0.9190 - val_loss: 0.4579 - val_accuracy: 0.8540
Epoch 4/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.2057 - accuracy: 0.9516
Epoc

In [8]:
# 위 코드를 통해 tensorflow checkpoint 파일을 만들고 epock가 종료될 때 마다 업데이트
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [9]:
# 훈련하지 않은 새로운 모델 만들기(정확도 10% 미만)
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.3995 - accuracy: 0.0690
훈련되지 않은 모델의 정확도:  6.90%


In [10]:
# checkpoint에서 가중치를 로드하고 다시 평가(저장된 값을 불러와 평가하니 87%)
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4278 - accuracy: 0.8680
복원된 모델의 정확도: 86.80%


In [16]:
# checkpoint callback 매개변수
# 이 콜백함수는 매개변수를 통해 checkpoint 이름을 고유하게 만들거나, 주기를 조정 할 수 있음.

# 파일 이름에 epoch 번호를 포함시켜 5번마다 고유한 이름으로 저장되도록
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 5번째 epoch마다 가중치 저장
    period=5)

model = create_model()
# model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [17]:
! ls {checkpoint_dir} # 만들어진 체크포인트 확인(50번 반복이니 10개 파일 생성)

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [18]:
latest = tf.train.latest_checkpoint(checkpoint_dir) # 마지막 체크포인트 선택
latest 
# tensorflow는 기본적으로 최근 5개의 checkpoint만 저장

'training_2/cp-0050.ckpt'

In [19]:
# 모델을 초기화하고 최근 checkpoint를 로드해 테스트
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5390 - accuracy: 0.8860
복원된 모델의 정확도: 88.60%


### checkpoint file

위 코드는 가중치를 checkpoint format의 file에 저장.   
이 file에는 훈련된 binary format의 가중치로
* 모델의 가중치를 포함하는 하나 이상의 샤드(shard)
* 가중치가 어느 샤드에 저장되어 있는지를 나타내는 index file

만일 단일머신에서 모델을 훈련한다면 .data-00000-of-00001 확장자를 가진 샤드 하나만 생성됨.   
지금은 단일머신 모델이 아니기 때문에 index file도 같이 생성되는 것.

In [20]:
# 수동으로 가중치 저장하기
# 위의 예제는 checkpoint를 통해 가중치를 저장하고 모델에 로드하는 방법
# 아래 예제는 수동으로 가중치를 저장하는 예제로 Model.save_weights 메서드 사용

model.save_weights('./checkpoints/my_checkpoint') # 가중치 저장

model = create_model() 
model.load_weights('./checkpoints/my_checkpoint') # 가중치 복원

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5390 - accuracy: 0.8860
복원된 모델의 정확도: 88.60%


In [21]:
# 모델 전체 저장하기
# 전체 모델을 하나에 저장 할 수 있음(가중치, 모델구성, optimizer 설정까지 포함)
# model의 checkpoint를 저장하므로 추후 원본 코드를 사용하지 않고도 정확히 동일한 상태에서 다시 훈련(학습) 시작 가능
# HDF5 파일로 저장하기(Hierarchical Data Format, 계층적 데이터 형식)
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장(keras는 HDF5 표준을 따르는 기본 저장 포맷을 제공)
# 저장된 모델을 하나의 binary blob처럼 다룰 수 있음
model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 366us/sample - loss: 1.1666 - accuracy: 0.6650
Epoch 2/5
1000/1000 [==============================] - 0s 86us/sample - loss: 0.4203 - accuracy: 0.8870
Epoch 3/5
1000/1000 [==============================] - 0s 83us/sample - loss: 0.2780 - accuracy: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 81us/sample - loss: 0.2230 - accuracy: 0.9410
Epoch 5/5
1000/1000 [==============================] - 0s 81us/sample - loss: 0.1651 - accuracy: 0.9560


In [22]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_10 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [23]:
# 정확도 확인
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))
# 초기 모델과 100% 일치
# 이 기법은 가중치, 모델 설정(구조), 옵티마이저까지 모두 저장

1000/1 - 0s - loss: 0.5618 - accuracy: 0.8640
복원된 모델의 정확도: 86.40%


In [24]:
# saved_model 사용하기
# 이 메서드는 실험적이므로 향후 변경될 가능성 있음
model = create_model()

model.fit(train_images, train_labels, epochs=5)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 428us/sample - loss: 1.1693 - accuracy: 0.6620
Epoch 2/5
1000/1000 [==============================] - 0s 104us/sample - loss: 0.4254 - accuracy: 0.8800
Epoch 3/5
1000/1000 [==============================] - 0s 92us/sample - loss: 0.2929 - accuracy: 0.9280
Epoch 4/5
1000/1000 [==============================] - 0s 87us/sample - loss: 0.2162 - accuracy: 0.9470
Epoch 5/5
1000/1000 [==============================] - 0s 86us/sample - loss: 0.1566 - accuracy: 0.9650


In [25]:
# export_saved_model 함수를 통해 timestamp를 이름으로 가진 디렉토리에 저장
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)
saved_model_path

Instructions for updating:
Please use `model.save(..., save_format="tf")` or `tf.keras.models.save_model(..., save_format="tf")`.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO:tensorflow:Signatures INCLUDED in export for Train: ['train']
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: None
INFO

'./saved_models/1586498519'

In [26]:
!ls saved_models/ # 저장된 모델의 목록 확인(unix time)

1586498519


In [27]:
# 저장된 모델로부터 새로운 keras 모델을 로드
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Instructions for updating:
The experimental save and load functions have been  deprecated. Please switch to `tf.keras.models.load_model`.
Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_11 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [30]:
# 복원된 모델을 실행해보면 젤 처음 지정한 사이즈 맞음
model.predict(test_images).shape

(1000, 10)

In [31]:
# 이 모델을 evaluate하려면 그전에 compile 해야함.
# 단지 저장된 모델의 배포라면 이 단계가 필요하지 않습니다.

new_model.compile(optimizer=model.optimizer, # 복원된 옵티마이저를 사용합니다.
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# 복원된 모델을 평가합니다
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5560 - accuracy: 0.8500
복원된 모델의 정확도: 85.00%
